# Setup

In [ ]:
# Update the settings to point to the datasets and runs directories
from ultralytics import settings
from ct_detector import DATASETS_DIR, ROOT_DIR
import os

settings.update({'datasets_dir': DATASETS_DIR, 'runs_dir': os.path.join(ROOT_DIR, 'runs')})

# Single model predictions

## Basic example

In [ ]:
# Import the necessary libraries and our custom predictor
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# Prepare the predictor and set up its parameters
analysis_pred = CtPredictor(
    overrides={
        "conf": 0.3,      # confidence threshold
        "save": False,    # don't save annotated images
        "save_txt": False, # no label files
    },
    handle_existing_labels="skip",  # not used since we aren't saving
    labels_dir="",
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = analysis_pred

# Now run predictions on a folder or an image. Stream =True is used to get results as they are processed in a form of a generator that we can iterate over as shown below.
results = model.predict(data_path, stream=True)

# results_single_analysis is a generator of Results. Let's just show five
counter = 0
for r in results:
    counter += 1
    if counter < 5:
        print(f"Image: {r.path}, #boxes={len(r.boxes)}")

## Prediction with a callback example

In [ ]:
# Import the necessary libraries and our custom predictor
import os
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# We import the default callback dictionary with keys for each step of the process.
# We also import an example callback that displays results.
from ultralytics.utils.callbacks import default_callbacks
from ct_detector.callbacks.base import display_results

# You can specify callbacks for each stage of the prediction process
predictor_callbacks = default_callbacks.copy()
predictor_callbacks['on_predict_batch_end'] = [display_results]

# Prepare the predictor and set up its parameters including the callbacks
analysis_pred = CtPredictor(
    overrides={
        "conf": 0.3,      # confidence threshold
        "save": False,    # don't save annotated images
        "save_txt": False, # no label files
        "verbose": False # no verbose output
    },
    _callbacks=predictor_callbacks
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = analysis_pred

# Alternatively we assign the callbacks to the predictor after loading the model
# model.predictor.callbacks = predictor_callbacks

# Now run predictions on a folder or an image. Stream =False is used to get results at once in a form of a list of Results. They are stored in memory and can be accessed via the predictor.results attribute. However, this is not recommended for large datasets.
results = model.predict(data_path, stream=False)

# results is a list of Results. Let's just show five
for r in results:
    print(f"Image: {os.path.basename(r.path)}, #boxes={len(r.boxes)}")

## Prediction with auto-annotation example

In [ ]:
# Import the necessary libraries and our custom predictor
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# Prepare the predictor and set up its parameters, this time allowing auto-annotation
auto_annot_pred = CtPredictor(
    overrides={
        "conf": 0.25,
        "save": True,           # save annotated images
        "save_txt": True       # save .txt label files
    },
    handle_existing_labels="rename",   # rename existing .txt
    labels_dir="custom_labels",        # store them in custom_labels/ folder
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = auto_annot_pred

# Now run predictions on a folder or an image
results = model.predict(data_path, stream=False)

## Prediction with individual tracking example

In [ ]:
# Import the necessary libraries and our custom predictor
import os
from ultralytics import YOLO
from ct_detector.model.predict import CtPredictor
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR

# Load a model
model_path = MODELS['eie_t_1_yolov8m']  # or your custom model: 'path/to/best.pt'

# Get a dataset
data_path = os.path.join(DATASETS_DIR, "2", "test", "images")  # or your dataset .yaml or folder of images

# We import the default callback dictionary with keys for each step of the process.
# We also import an example callback that displays results.
from ultralytics.utils.callbacks import default_callbacks
from ct_detector.callbacks.track import track_results
from ct_detector.callbacks.base import display_results
from ct_detector.model.track import CtTracker

# Define the tracker we will use for predictions. If you want to keep the tracking memory from previous prediction run (for example when running predictions by folder)
# do not re-initiate the tracker but keep it defined from previous runs.
tracker = CtTracker()
track = track_results(tracker, persist=True) # persist True to keep memory between frames

# You can specify callbacks for each stage of the prediction process
predictor_callbacks = default_callbacks.copy()
predictor_callbacks['on_predict_batch_end'] = [track, display_results]

# Prepare the predictor and set up its parameters including the callbacks
analysis_pred = CtPredictor(
    overrides={
        "conf": 0.3,      # confidence threshold
        "save": False,    # don't save annotated images
        "save_txt": False, # no label files
        "verbose": False # no verbose output
    },
    _callbacks=predictor_callbacks
)

# Load the model and assign the predictor
model = YOLO(model_path)
model.predictor = analysis_pred

# Alternatively we assign the callbacks to the predictor after loading the model
# model.predictor.callbacks = predictor_callbacks

# Now run predictions on a folder or an image. Stream =False is used to get results at once in a form of a list of Results. They are stored in memory and can be accessed via the predictor.results attribute. However, this is not recommended for large datasets.
results = model.predict(data_path, stream=True)

# results is a list of Results. Let's just show five
for r in results:

    print(f"Image: {os.path.basename(r.path)}, #boxes={len(r.boxes)}")
    if r and hasattr(r, "boxes"):
        if r.boxes:
            print(f"Tracked successfully: {r.boxes.is_track}")
            print(f"Detected individual IDs: {[str(int(ind)) for ind, cls in zip(list(r.boxes.id), list(r.boxes.cls)) if all([ind > -1, int(cls) == 2 or int(cls) == 0])] if r.boxes.id is not None else 'None identified'}")

# Multi-model predictions

## Basic example

In [ ]:
from ct_detector.model.ensemble import CtEnsembler
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os

# 1) Get a dataset
data_path = os.path.join(DATASETS_DIR, "1", "val.txt")  # or your dataset .yaml or folder of images

# 2) Create the ensemble
ensembler = CtEnsembler(
    model_paths=[MODELS['eie_t_1_yolov8m'], MODELS['eie_t_1_yolov9m']], # Specify a list of models
    predictor_overrides={"conf": 0.3, "save_txt": False, "verbose": False},  # e.g. no partial labels
)

# 3) Run predictions on a .txt of images
gen = ensembler.predict(
    source=data_path,
    nms_iou_thres=0.5, # threshold for overlap during result merging
    nms_conf_thres=0.3, # threshold of conf considered during result merging
    class_agnostic=False, # whether to consider boxes of any class for merging in case of sufficient overlap or whether only consider same class
    class_merge_map={0:0, 1:1, 2:0, 3:2}  # Merge classes 0 and 2. Specify groups of classes that are together considered for merging.
)

# 3) For each merged frame, do more logic if you want
for idx, merged_frame in enumerate(gen):
    # Possibly do final .txt saving, or display bounding boxes
    # merged_frame.save_txt(f"ensemble_labels/frame_{idx}.txt")
    pass

## Ensembled prediction with callbacks example

In [ ]:
from ct_detector.model.ensemble import CtEnsembler
from ct_detector.model import MODELS, DATASETS, DATASETS_DIR
import os

# Create the ensemble
ensembler = CtEnsembler(
    model_paths=[MODELS['eie_t_1_yolov8m'], MODELS['eie_t_1_yolov9m']], # Specify a list of models
    predictor_overrides={"conf": 0.3, "save_txt": False, "verbose": False},  # e.g. no partial labels
)

# Import callbacks that print data about made predictions and display results
from ct_detector.callbacks.database import print_prediction_data
from ct_detector.callbacks.base import display_results

# Run predictions on a .txt of images with custom callbacks
gen = ensembler.predict(
    source=data_path,
    nms_iou_thres=0.5, # threshold for overlap during result merging
    nms_conf_thres=0.3, # threshold of conf considered during result merging
    class_agnostic=False, # whether to consider boxes of any class for merging in case of sufficient overlap or whether only consider same class
    class_merge_map={0:0, 1:1, 2:0, 3:2},  # Merge classes 0 and 2. Specify groups of classes that are together considered for merging.
    _callbacks=[print_prediction_data, display_results] # A list of callback to execute after a result is merged.
)

# For each merged frame, do more logic if you want
for idx, merged_frame in enumerate(gen):
    # Possibly do final .txt saving, or display bounding boxes
    # merged_frame.save_txt(f"ensemble_labels/frame_{idx}.txt")
    pass